In [1]:
from IPython.display import display

from typing import Dict

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [2]:
# !pip install matplotlib

## Configs

In [3]:
nGPUs = (1,2,4)

## Read data

In [4]:
def read_csv(f_name: str) -> pd.DataFrame:
    try:
        _df = pd.read_csv(f_name, skiprows=3)
        # print("%s UoM: "%f_name, _df.iloc[0])
        _df = _df.drop(0)
        _df = _df.astype({'Start': float, 'Duration': float})
    except:
        _df = pd.read_csv(f_name, skiprows=3, skipfooter=1)
        # print("%s UoM: "%f_name, _df.iloc[0])
        _df = _df.drop(0)
        _df = _df.astype({'Start': float, 'Duration': float})
    return _df

def get_f_name(n_gpu: int) -> str:
    return "profiler_wangchanberta_bs16_gpu%d.csv"%n_gpu



## Analysis

In [5]:
def get_ins_group(ops: str) -> str:
    if "gemm" in ops:
        return "matrix-mul"
    elif "CUDA memcpy" in ops or "nccl" in ops.lower() or "copy_device_to_device" in ops.lower()\
        or "CUDA memset" in ops:
        return "memory_mgmt"
    elif "::native" in ops or "vectorized_elementwise" in ops or "_cpp1_ii" in ops or "reduce_kernel" in ops:
        return "custom_ops"
    return "other"

def aggregate(df: pd.DataFrame) -> None:
    df["ops_group"] = df.apply(lambda x: get_ins_group(x.Name), axis=1)
    
    called_cuda_interfaces = df["Name"].unique().tolist()
    memory_interfaces = list(filter(lambda x: 'CUDA mem' in x, called_cuda_interfaces))
    memory_interfaces.append('ncclBroadcastRingLLKernel_copy_i8(ncclColl)')
    # Elapsed time
    print("--------- Elapsed time (ms): ", df["Start"].max() - df["Start"].min())
    # devices
    print("--------- devices ")
    display(df["Device"].unique().tolist())
    # memory mgmt
    print("--------- memory mgmt ")
    for mi in memory_interfaces:
        print(mi)
        display(df.query("Name == '%s'"%mi).groupby('Device')["Duration"].sum().reset_index())
    display(df[df["Name"].isin(memory_interfaces)].groupby('Device')["Duration"].sum().reset_index())
    # Top N
    print("--------- Top N ")
    display(df.groupby(['Device', 'Name'])["Duration"].sum().nlargest(10).reset_index())
    display(df.groupby(['Device', 'ops_group'])["Duration"].sum().reset_index())
    # Aggregate
    print("--------- Aggregate ")
    agg_result = df.groupby('ops_group')["Duration"].sum().reset_index()
    display(agg_result)
    agg_result["Percent"] = agg_result[["Duration",]].apply(lambda x: 100*x/x.sum())
    display(agg_result)

def analyze(n_gpus: int = nGPUs) -> None:
    for n_gpu in n_gpus:
        print("=========== Start %d GPUs ==========="%n_gpu)
        f_name = get_f_name(n_gpu)
        df = read_csv("profiler_wangchanberta_bs16_gpu%d.csv"%n_gpu)
        aggregate(df)
        print("=========== End %d GPUs =========== \n\n\n"%n_gpu)

## Check Unique GPUs

In [6]:
analyze((1,2,4))

=========== Start 1 GPUs ===========


/tmp/ipykernel_20599/2847321564.py:43: DtypeWarning: Columns (0,1,9,10,11,12) have mixed types.Specify dtype option on import or set low_memory=False.
  df = read_csv("profiler_wangchanberta_bs16_gpu%d.csv"%n_gpu)


--------- Elapsed time (ms):  2110.694652
--------- devices 


['Tesla V100-SXM2-32GB-LS (0)']

--------- memory mgmt 
[CUDA memcpy HtoD]


...

,Device,ops_group,Duration
0,Tesla V100-SXM2-32GB-LS (0),custom_ops,120595.109126
1,Tesla V100-SXM2-32GB-LS (0),matrix-mul,393645.783055
2,Tesla V100-SXM2-32GB-LS (0),memory_mgmt,286238.803875
3,Tesla V100-SXM2-32GB-LS (0),other,328.071178
4,Tesla V100-SXM2-32GB-LS (1),custom_ops,105301.398673
5,Tesla V100-SXM2-32GB-LS (1),matrix-mul,446267.405010
6,Tesla V100-SXM2-32GB-LS (1),memory_mgmt,285155.661257
7,Tesla V100-SXM2-32GB-LS (1),other,322.199943
8,Tesla V100-SXM2-32GB-LS (2),custom_ops,53997.735935
9,Tesla V100-SXM2-32GB-LS (2),matrix-mul,227996.473690


--------- Aggregate 


,ops_group,Duration
0,custom_ops,3.837383e+05
1,matrix-mul,1.501899e+06
2,memory_mgmt,1.147016e+06
3,other,1.270684e+03


,ops_group,Duration,Percent
0,custom_ops,3.837383e+05,12.648253
1,matrix-mul,1.501899e+06,49.503512
2,memory_mgmt,1.147016e+06,37.806352
3,other,1.270684e+03,0.041883


=========== End 4 GPUs =========== 



